In [13]:
#A simple DNN model of three hidden layer is developed in this code
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization,Activation
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.model_selection import StratifiedKFold
import pandas as pd
from keras.optimizers import SGD
from keras.optimizers import Adam
import tensorflow as tf
import keras

In [3]:
#PCA transformation
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np 

my_data = np.genfromtxt('data.row/ASDDataset_Augmented.csv', delimiter=',')
X = my_data[1:,0:10000]
labels = my_data[1:,10000].astype(dtype='int')

df = pd.DataFrame(my_data)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(df.mean(), inplace=True)

nDim = 50
pca = PCA(n_components=nDim)
X_transformed = pca.fit(X).transform(X)

In [4]:
#print(dataset.shape)
X = X_transformed
Y =  my_data[1:,10000].astype(dtype='int')
#print(labels)

In [27]:
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=0).split(X, Y)
cvscores = []
for train, test in kfold:
    
    # تعریف مدل
    model = Sequential()
    model.add(Dense(400, input_dim=(50), activation='relu'))
    
    model.add(Dense(200, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dropout(0.2))

    #model.add(Dense(200, input_dim=(50),activation='relu'))
    #model.add(BatchNormalization())
    #model.add(Activation(activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    
    
    # Compile model
    
    #opt = SGD(lr=0.01,momentum=0.8,nesterov=False)
    #opt = Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,decay=0.0,epsilon=1e-07,amsgrad=False)
    opt = Adam(learning_rate=0.002,decay = 1e-6)
    model.compile(loss= 'binary_crossentropy' , optimizer= opt , metrics=[ 'accuracy' ])
    
    # Fit the model
    model.fit(X[train], Y[train], validation_split=0.2,  epochs=100, batch_size=12, verbose=0)
    
   
    # ارزیابی مدل
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print(scores)
    
    
    y_pred = model.predict(X[test])
    AUC = roc_auc_score(Y[test], y_pred.round())
print(AUC)

[0.557946503162384, 0.729468584060669]
[0.5833921432495117, 0.7137330770492554]
[0.5448030233383179, 0.7243713736534119]
0.6729323308270677
